In [ ]:
#修士論文のコードを本当はアップロードしたかったのですが、修士論文が共同研究だった関係上守秘義務が発生し
#守秘義務が2027年まで残っているので、やむなくChatGPTに
#「修士論文で使用したコードを就活のポートフォリオとして掲載したいですが、修士論文のコードは守秘義務が課せられており使用できません。
#そこで、以下のコードを守秘義務的に問題がないように、やりたいことは同じまま改変してください。」
#と説明して変換してもらいました。全セルを同時に入力すると超絶省略した形でしか変換してくれなかったので各セルごとに変換してもらいましたが、
#代わりに毎回importする超絶冗長コードになっている部分があります。
#ですので、「なんかこういうことをやっていたんだなー」程度の雰囲気で読んでいただけると幸いです。

#簡単に全体感を述べると、
#1. データ形式の乱れの処理を行いました。
#2. データを可視化してみてどう分析するかを考えました(これはこの後も適時行なっていました)。
#3. 異常値の切り捨てを行いました。
#4. 私が研究したデータは時系列データで、時系列的な欠損の補完や、正規分布への変換等を行いました。具体的には対数変換やYeo-Johnson変換を試しました。正規分布の確認はShapiro-WilkテストとKolmogorov-Smirnovテストを実施しています。
#5. 特徴量間で高すぎる相関を示すものは取り除きました。
#6. XGBoostで予測を行い、特徴量の重要度を求めました。
#7. 特徴量の重要度に基づいて、AUCが最大になる特徴量の組み合わせを求めました。
#他にもPyCaretを用いて様々な機械学習手法を同時に比較検討してみたり、LSTMで時系列方向での予測を行ったりしました。
#そもそも時系列データなのになんでLSTM等の時系列手法ではなくXGBoostを使っているんだ？という疑問が湧くと思いますが、
#1つ目の理由が、患者データの計測感覚が1日1回だったため前の状態が次の状態に与える影響の法則性が小さく時系列方向での周期性も見られにくかったため
#2つ目の理由が、リアルワールドデータではありがちかもしれませんが、計測忘れによる空データに対応する必要があったためです。

In [ ]:
import numpy as np
import pandas as pd
import copy
import datetime as dt
import warnings

warnings.filterwarnings('ignore')

# データ読み込みのためのパス設定
data_file_path1 = 'path/to/your/datafile1.csv'
data_file_path2 = 'path/to/your/datafile2.csv'

# pandasを使用してCSVデータを読み込み
data1 = pd.read_csv(data_file_path1, encoding='utf-8', sep=",", parse_dates=[1])  # データ1の読み込み
data2 = pd.read_csv(data_file_path2, encoding='utf-8', sep=",", parse_dates=[-6])  # データ2の読み込み

# データの深いコピーを作成
data1_original = copy.deepcopy(data1)

# 不要な列の削除と列名の変更
data1 = data1.drop(['column1', 'column2', 'column3', 'column4', 'column5'], axis=1)
data1 = data1.rename(columns={'old_name':'user'})

# 新しい列の計算と追加
data1['new_metric'] = data1['metric1'] / data1['metric2']
data1['date'] = data1['datetime_column'].apply(lambda x: dt.date(x.year, x.month, x.day))
data1['time'] = data1['datetime_column'].apply(lambda x: dt.time(x.hour, x.minute, x.second).strftime('%H:%M:%S'))
data1['time'] = pd.to_datetime(data1['time']).dt.hour * 60 + pd.to_datetime(data1['time']).dt.minute
data1['date'] = pd.to_datetime(data1['date'])
data1['birth_date'] = pd.to_datetime(data1['birth_date'])
data1['age'] = (data1['date'] - data1['birth_date']).astype('timedelta64[Y]')

# ユーザー名の誤りを修正
data1 = data1.replace({'user': {'incorrect_name1': 'correct_name1'}})
data1 = data1.sort_values(['user', 'date']).reset_index(drop=True)

# データの選択基準に基づくフィルタリング
data1_filtered = copy.deepcopy(data1)
data1_quality = copy.deepcopy(data1)
criteria = (data1_filtered['quality_metric'] >= 90) & \
           (data1_filtered['metric3'] < 63) & \
           (data1_filtered['metric4'] < 100) & \
           ...  # 他の条件も同様に追加
data1_filtered = data1_filtered[criteria]

# 質の高いデータのみを選択
data1_quality['quality_flag'] = 0
data1_quality.loc[data1_filtered.index, 'quality_flag'] = 1

# 不要な列の削除
data1_filtered = data1_filtered.drop('metric5', axis=1)
data1_quality = data1_quality.drop('metric5', axis=1)

# 質の高いデータセットを特定
data1_high_quality = data1[data1['quality_metric'] == 100].drop('metric5', axis=1)

# pandasの設定変更
pd.set_option('display.max_rows', 500)

In [ ]:
# 各ユーザーの各日付について最も品質が高いデータの中から平均的なデータのみを選択
# 実行に時間がかかる可能性があります。
# 一時的なデータセットにコピーしておきます。必要に応じて他のデータセットを使用できます。
temp_dataset = copy.deepcopy(data_high_quality)
user_list = temp_dataset['user'].unique()
iteration_count = 0
for user in user_list:
    user_data = temp_dataset[temp_dataset['user'] == user]
    date_list = user_data['date'].unique()
    for date in date_list:
        date_data = user_data[user_data['date'] == date]  # 各ユーザーの各日に測定したデータを選択
        max_quality = date_data['quality_metric'].max()  # その日の最大品質値
        max_quality_data = pd.DataFrame(date_data[date_data['quality_metric'] == max_quality])
        if max_quality_data.shape[0] >= 1:
            diff = (max_quality_data['metric3'] - max_quality_data['metric3'].mean()).abs()
            max_quality_data = max_quality_data.loc[[diff.idxmin()]]  # 最も平均に近いデータを選択
        if iteration_count == 0:
            selected_data = max_quality_data
        else:
            selected_data = pd.concat([selected_data, max_quality_data])
        iteration_count += 1

selected_data.reset_index(drop=True, inplace=True)


In [ ]:
import seaborn as sns

# データの可視化：特定の数値データの分布をヒストグラムで表示
sns.histplot(data=data_filtered, x='numeric_feature')
plt.title('Distribution of Numeric Feature')
plt.xlabel('Numeric Feature')
plt.ylabel('Frequency')
plt.show()


In [ ]:
#このコードは、ユーザーごとに日付の範囲を生成し、元のデータセットにマージして、
#欠損日も含めた完全な時系列データセットを作成するためのものです。
#具体的な変数名やデータセット名を一般化し、データの整形処理を行う方法を示します。
import pandas as pd
import copy

# データの準備
data_prepared = copy.deepcopy(data_filtered)

# 各ユーザーの測定開始日と最終日を取得
date_start = data_prepared.groupby('user')['date'].min()
date_end = data_prepared.groupby('user')['date'].max()

# 全ユーザーの日付範囲を生成して1つのデータフレームに集約
date_list_all_users = pd.DataFrame(index=[], columns=['date', 'user'])
for user_id in date_start.index:
    date_range_each_user = pd.DataFrame(pd.date_range(start=date_start[user_id], end=date_end[user_id], freq='D'), columns=["date"])
    date_range_each_user['user'] = user_id
    date_list_all_users = date_list_all_users.append(date_range_each_user, ignore_index=True)

# 元のデータセットとマージして、欠損日も含む完全な時系列データセットを作成
data_complete_timeseries = pd.merge(date_list_all_users, data_prepared, on=['date', 'user'], how='left')

# 不要な列の削除
data_complete_timeseries = data_complete_timeseries.drop(['datetime_column', 'birth_date_column'], axis=1)


#この改変により、特定のプロジェクトやデータセットに依存する部分を抽象化し、
#一般的なデータ処理のフローを示しています。data_filtered は前処理済みのデータセット、
#data_complete_timeseries は欠損日を含む完全な時系列データセットを表しています。
#具体的な列名（例えば datetime_column や birth_date_column）は、実際のデータセットの内容に合わせて適宜置き換えてください。

In [ ]:
#このコードは、データセットに対して対数変換を適用し、特定の測定値の分布を正規化するプロセスを示しています。
#さらに、各ユーザーごとに日付の範囲を生成し、元のデータセットにマージして完全な時系列データセットを作成します。
#具体的な変数名やデータセット名を一般化し、データの変換と整形処理を行う方法を示します。

import pandas as pd
import numpy as np
import copy

# データの準備
data_processed = copy.deepcopy(data_complete_timeseries)

# 不要な列の削除
data_processed = data_processed.drop('quality_metric', axis=1)

# 対数変換を適用する列名のリスト
metrics_to_transform = ['metric_a', 'metric_b', 'metric_c', 'metric_d', 'metric_e', 'metric_f', 'metric_g', 'metric_h', 'metric_i']

# 対数変換の適用
for metric in metrics_to_transform:
    data_processed['Ln_' + metric] = np.log(data_processed[metric] + 1)  # 0を避けるために+1
    data_processed = data_processed.drop(metric, axis=1)

# 特定の条件に基づくデータのフィルタリング
filtered_criteria = (data_processed['Ln_metric_a'] < 5.5) & \
                    (data_processed['Ln_metric_b'] < 5.5) & \
                    ...  # 他の条件も同様に追加
data_processed = data_processed[filtered_criteria]

# 各ユーザーの測定開始日と最終日を取得
date_start = data_processed.groupby('user')['date'].min()
date_end = data_processed.groupby('user')['date'].max()

# 全ユーザーの日付範囲を生成して1つのデータフレームに集約
date_list_all_users = pd.DataFrame(index=[], columns=['date', 'user'])
for user_id in date_start.index:
    date_range_each_user = pd.DataFrame(pd.date_range(start=date_start[user_id], end=date_end[user_id], freq='D'), columns=["date"])
    date_range_each_user['user'] = user_id
    date_list_all_users = date_list_all_users.append(date_range_each_user, ignore_index=True)

# 元のデータセットとマージして、欠損日も含む完全な時系列データセットを作成
data_with_all_dates = pd.merge(date_list_all_users, data_processed, on=['date', 'user'], how='left')

#このコードは、特定の測定値に対数変換を適用し、その後でデータセットを特定の条件に基づいてフィルタリングします。
#また、各ユーザーについて可能なすべての日付を含むデータフレームを生成し、これを元のデータセットにマージしています。
#これにより、欠損値をNaNとして含む完全な時系列データセットが作成されます。
#実際に使用する際には、metrics_to_transform のリストやフィルタリング条件を実際のデータに合わせて調整してください。

In [ ]:
#このコードは、時系列データに対して過去数日間の情報を特徴量として組み込む処理と、
#その後のデータの集約（平均や標準偏差の計算）を行うプロセスを示しています。
#具体的な変数名やデータセット名を一般化し、データの変換と集約処理を行う方法を示します。

import pandas as pd
import numpy as np
import copy

# 特徴量リストの定義
feature_list = pd.Series(['feature_a', 'Ln_feature_b', 'Ln_feature_c', 'Ln_feature_d', 'Ln_feature_e', 'Ln_feature_f', 'Ln_feature_g',
       'Ln_feature_h', 'Ln_feature_i', 'Ln_feature_j', 'Ln_feature_k'])

# ユーザーごとのデータカウントと特定の条件を満たすユーザーの選定
user_days_count = data_with_all_dates.groupby(['user'])['Ln_feature_b'].count()
users_with_enough_data = user_days_count[user_days_count >= 8].index.tolist()

# 過去のデータを特徴量として組み込む処理
multidays = 7
for user_id in users_with_enough_data:
  user_data = data_with_all_dates[data_with_all_dates['user'] == user_id]
  for j in range(1, multidays):
    shifted_data = user_data[feature_list].shift(j).add_suffix('_' + str(j) + 'day(s)_before')
    user_data = pd.concat([user_data, shifted_data], axis=1)

# 集約処理：各特徴量に対する平均と標準偏差の計算
for feature in feature_list:
  aggregated_features = user_data.filter(regex=feature).copy()
  user_data[feature + '_average'] = aggregated_features.mean(axis=1)
  user_data[feature + '_std'] = aggregated_features.std(axis=1)
  user_data[feature + '_cv'] = user_data[feature + '_std'] / user_data[feature + '_average']

# 特定の文字列を含む/含まない列名を取得し、データをフィルタリング
character = 'before'
filtered_columns = [col for col in user_data.columns if character not in col]
data_aggregated = user_data[filtered_columns]

In [ ]:
data_aggregated

In [ ]:
#このコードは、あるデータセットに対して状態を数値コードに置き換え、
#特定のカテゴリーに関連する列を除外し、さらにユーザーごとに日付順にソートする処理を行い、
#最後に各ユーザーの平均値を基準にデータを正規化するプロセスを示しています。
#具体的な変数名やデータセット名を一般化し、データの前処理と変換処理を行う方法を示します。
import pandas as pd
import numpy as np
import copy

# データの準備と前処理
data_prepared = copy.deepcopy(data)

# 状態を数値コードに置き換え
data_prepared = data_prepared.replace({'status_column': {'status1': 0, 'status2': 1, 'status3': 2, 'status4': 3, 'status5': 4}})
data_prepared = data_prepared.drop(['status2', 'status3', 'status4', 'status5', 'extra_column'], axis=1)

# 特定の文字列を含む列の除去
exclude_character = 'exclude_term'
columns_to_include = [col for col in data_prepared.columns if exclude_character not in col]
data_prepared = data_prepared[columns_to_include]

# 選択された列でデータセットをフィルタリング
selected_columns = ['user_id', 'date', 'status_column', 'metric1', 'metric2', 'metric3', 'metric4']
data_selected = data_prepared[selected_columns]

# ユーザーごとに日付順にソート
data_selected_sorted = data_selected.sort_values(['user_id', 'date'])

# 各ユーザーごとに平均値を基準にデータを正規化
for user_id in data_selected_sorted['user_id'].unique():
    user_data = data_selected_sorted[data_selected_sorted['user_id'] == user_id]
    mean_values = user_data.mean(axis=0)
    for col in ['metric1', 'metric2', 'metric3', 'metric4']:
        user_data[col] = user_data[col] - mean_values[col]
    if 'normalized_data' in locals():
        normalized_data = pd.concat([normalized_data, user_data])
    else:
        normalized_data = user_data

normalized_data = normalized_data.reset_index(drop=True)

# 結果の表示や保存処理
# display(normalized_data.head())
# normalized_data.to_csv('path/to/your/normalized_dataset.csv', index=False)


In [ ]:
#このコードは、データの読み込み、特定の列の削除、および特定の条件に基づいて列を選択する処理を行うスニペットです。
#データセットの名前や操作を一般化し、データの読み込みから列の選択までのプロセスを示します。
import pandas as pd
import numpy as np
import datetime as dt

# データファイルのパス（一般化）
data_file_path = 'path/to/your/processed_data.csv'

# pandasを使用してデータを読み込み
processed_data = pd.read_csv(data_file_path, encoding='utf-8', sep=",", parse_dates=[1])

# 特定の列を削除（一般的な列名に置き換え）
processed_data = processed_data.drop(['unnecessary_column1', 'datetime_column', 'birth_date_column'], axis=1)

# 時間を数値に変換
processed_data['time_numeric'] = pd.to_datetime(processed_data['time_column']).dt.hour * 60 + pd.to_datetime(processed_data['time_column']).dt.minute

# 特定の値の置換（一般化）
processed_data = processed_data.replace({'user_column': {'incorrect_value1': 'correct_value1'}})

# 特定の文字列を含む/含まない列の選択と除去
exclude_terms = ['exclude_term1', 'exclude_term2', 'exclude_term3', 'exclude_term4', 'exclude_term5']
for term in exclude_terms:
    columns_to_exclude = [col for col in processed_data.columns if term in col]
    processed_data = processed_data.drop(columns_to_exclude, axis=1)

# 必要な列のみを選択（一般化）
required_columns = ['date', 'user_column', 'time_numeric', 'age_column']
processed_data = processed_data[required_columns + [col for col in processed_data.columns if 'Ln_' in col]]

# データをユーザーと日付でソート
processed_data = processed_data.sort_values(['user_column', 'date'])

# 結果の表示や保存処理
# display(processed_data.head())
# processed_data.to_csv('path/to/your/final_dataset.csv', index=False)


In [ ]:
#このコードは、特定のデータセットから選択された特徴量を組み合わせて新しいデータセットを作成し、
#その後にデータの前処理を行うプロセスを示しています。
#具体的な変数名やデータセット名を一般化し、データの結合、欠損値の処理、カテゴリカル変数の数値化を行う方法を示します。
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder

# データの準備と結合
feature_data = copy.deepcopy(processed_data)  # processed_dataは前のステップで前処理されたデータセット
target_data = target_dataset[['user_id', 'date', 'target_variable']]  # target_datasetは目的変数を含むデータセット

# データセットの結合
merged_dataset = pd.merge(left=target_data, right=feature_data, how='left', on=['user_id', 'date'])

# ユーザーごとの年齢情報を結合
user_age_info = feature_data.groupby('user_id')['age'].apply(lambda x: x.mode()).reset_index()
user_age_info = user_age_info[user_age_info['level_1'] == 0][['user_id', 'age']].set_index('user_id')
merged_dataset = pd.merge(merged_dataset, user_age_info, how='left', on='user_id')
merged_dataset['age'] = merged_dataset['age'].fillna(merged_dataset['age_y']).drop('age_y', axis=1)

# データの整理
merged_dataset.drop('date', axis=1, inplace=True)

# ユーザーIDの数値化
label_encoder = LabelEncoder()
merged_dataset['user_id_encoded'] = label_encoder.fit_transform(merged_dataset['user_id'])
merged_dataset.drop('user_id', axis=1, inplace=True)

# 欠損値の処理
merged_dataset.dropna(subset=['target_variable'], inplace=True)

# 数値特徴量のリスト作成
numeric_features = [col for col in merged_dataset.columns if col != 'target_variable']

# pandasの設定変更
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 最終的なデータセットの準備
final_dataset = copy.deepcopy(merged_dataset)


In [ ]:
#このコードは、データセットにYeo-Johnson変換を適用し、
#その後に統計モデルを用いて変換後のデータセットの変数間の関係を分析するプロセスを示しています。
#Yeo-Johnson変換は、データの正規性を改善するために使用されます。
#具体的な変数名やデータセット名を一般化し、データの変換と分析処理を行う方法を示します。
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
import statsmodels.api as sm

# データの準備
features_data = copy.deepcopy(features_dataset)  # features_datasetは特徴量を含むデータセット
target_data = copy.deepcopy(target_dataset)  # target_datasetは目的変数を含むデータセット

# 無限大の値をNaNに置き換え
features_data.replace([np.inf, -np.inf], np.nan, inplace=True)
target_data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Yeo-Johnson変換の適用
transformer = PowerTransformer(method='yeo-johnson', standardize=True)
scaler = MinMaxScaler()

for column in features_data.columns[2:]:
    transformed_data = scaler.fit_transform(features_data[[column]].dropna())
    transformed_data = transformer.fit_transform(transformed_data)
    features_data[column] = transformed_data

for column in target_data.columns[:-3]:
    transformed_data = scaler.fit_transform(target_data[[column]].dropna())
    transformed_data = transformer.fit_transform(transformed_data)
    target_data[column] = transformed_data

# 統計モデルによる分析
pvalues = pd.DataFrame(index=features_data.columns[2:], columns=target_data.columns[:-3])
for target_col in target_data.columns[:-3]:
    for feature_col in features_data.columns[2:]:
        merged_data = pd.merge(target_data[['user_id', 'date', target_col]],
                               features_data[['user_id', 'date', feature_col]],
                               on=['user_id', 'date'], how='inner').dropna()
        X = sm.add_constant(merged_data[feature_col])
        Y = merged_data[target_col]
        model = sm.OLS(Y, X).fit()
        pvalues.loc[feature_col, target_col] = model.pvalues[1]

# pvaluesの表示や分析
# display(pvalues)

In [ ]:
#このコードは、変換後のデータに対して統計的分析を行い、
#特定の変数間の関係性を評価し、残差の正規分布を検証するプロセスを示しています。
#具体的な変数名やデータセット名を一般化し、データの分析と残差の検証を行う方法を示します。
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

# 分析対象の目的変数と説明変数を選択
target_variable = 'target_metric'  # 例: 'target_metric' は分析したい目的変数
explanatory_variable = 'transformed_feature'  # 例: 'transformed_feature' は分析したい説明変数

# 分析用のデータセットの準備
analysis_dataset = pd.merge(target_data_subset, transformed_feature_data, how='left', on=['user_id', 'date'])

# 年齢情報の統合
user_age_info = transformed_feature_data.groupby('user_id')['age'].apply(lambda x: x.mode()).reset_index()
user_age_info = user_age_info[user_age_info['level_1'] == 0].set_index('user_id')
analysis_dataset = pd.merge(analysis_dataset, user_age_info, on='user_id', how='left')
analysis_dataset['age'] = analysis_dataset['age'].fillna(analysis_dataset['age_y']).drop(['age_y'], axis=1)

# 不要な列の削除
analysis_dataset = analysis_dataset.drop(['date', 'user_id'], axis=1).dropna()

# 統計モデルのフィッティング
X = sm.add_constant(analysis_dataset[explanatory_variable])
Y = analysis_dataset[target_variable]
model = sm.OLS(Y, X).fit()

# 結果の表示
print(model.summary())

# 回帰プロットと残差の正規性検定
sns.jointplot(data=analysis_dataset, x=explanatory_variable, y=target_variable, kind='reg')
plt.show()

plt.hist(model.resid, bins=40)
plt.title('Residuals Distribution')
plt.show()

fig = plt.figure(figsize=(8, 8))
stats.probplot(model.resid, dist="norm", plot=plt)
plt.title('Q-Q Plot')
plt.show()

# 残差の正規性検定
shapiro_test = stats.shapiro(model.resid)
ks_test = stats.kstest(model.resid, "norm")
print("Shapiro-Wilk Test:", shapiro_test)
print("Kolmogorov-Smirnov Test:", ks_test)

#この改変により、目的変数と説明変数の関係性を評価し、残差の正規性を検証するための統計的手法を適用しています。
#Yeo-Johnson変換後の変数を用いて統計モデルを構築し、回帰プロット、残差分布、Q-Qプロットを用いて分析結果を視覚化し、
#残差の正規性についてShapiro-WilkテストとKolmogorov-Smirnovテストを実施しています。

In [ ]:
#このコードは、XGBoostを用いて特定の目的変数に対する二値分類を行うためのデータ準備プロセスを示しています。
#具体的な変数名やデータセット名を一般化し、データの準備からモデルのトレーニングまでのプロセスを示します。
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb

# 目的変数を設定
target_variable = 'binary_outcome'  # 例えば、'binary_outcome'は分析したい二値の目的変数

# 分析用データセットの準備
merged_data = pd.merge(left=pro_target_data, right=transformed_features_data, how='left', on=['user_id', 'date'])

# ユーザーごとの年齢情報を結合
user_age_info = transformed_features_data.groupby('user_id')['age'].apply(lambda x: x.mode()).reset_index()
user_age_info = user_age_info[user_age_info['level_1'] == 0].set_index('user_id')
merged_data = pd.merge(merged_data, user_age_info, on='user_id', how='left')
merged_data['age'] = merged_data['age'].fillna(merged_data['age_y']).drop(['age_y'], axis=1)

# 不要な列の削除と欠損値の処理
merged_data = merged_data.drop(['date', 'user_id'], axis=1).replace([np.inf, -np.inf], np.nan).dropna()

# 目的変数の二値化
merged_data[target_variable] = (merged_data[target_variable] > 0).astype(int)

# データセットをトレーニングセットとテストセットに分割
X = merged_data.drop(target_variable, axis=1)
y = merged_data[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoostモデルのトレーニング
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgb_model.fit(X_train, y_train)

# トレーニング完了後、X_testを使用して予測を行い、性能を評価することが可能

In [ ]:
#このコードは、データセット内の特定の目的変数の分布をヒストグラムで視覚化するためのスニペットです。
#具体的な変数名を一般化し、データの可視化を行う方法を示します。
import seaborn as sns
import matplotlib.pyplot as plt

# 目的変数の分布をヒストグラムで表示
sns.histplot(data=processed_dataset, x='binary_outcome')
plt.title('Distribution of Binary Outcome')
plt.xlabel('Binary Outcome')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#このコードは、XGBoost分類器を用いた二値分類問題の解決と、結果の評価を行うプロセスを示しています。
#具体的な変数名やデータセット名を一般化し、モデルのトレーニングと評価を行う方法を示します。
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# 特徴量と目的変数を設定
features = analysis_dataset.drop('binary_outcome', axis=1)
outcome = analysis_dataset['binary_outcome']

# データセットをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(features, outcome, test_size=0.25, random_state=42)

# XGBoost分類器のトレーニング
xgb_classifier = XGBClassifier(tree_method='gpu_hist', max_depth=4, learning_rate=0.01)
xgb_classifier.fit(X_train, y_train)

# テストセットに対する予測確率の計算
probabilities = xgb_classifier.predict_proba(X_test)[:, 1]

# ROC AUCスコアの計算
auc_score = roc_auc_score(y_test, probabilities)

# ROCカーブの描画
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, probabilities)

plt.figure(figsize=(12, 7))
plt.plot(false_positive_rate, true_positive_rate, label=f'AUC (XGBoost) = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], 'k--', label='Random Chance')
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()


In [ ]:
#このコードは、XGBoost分類器を用いた機械学習モデルのトレーニングと、
#モデルの特徴量重要度の可視化を行うプロセスを示しています。
#具体的な変数名やデータセット名を一般化し、モデルのトレーニングと特徴量の重要度の評価を行う方法を示します。
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 目的変数の設定
binary_target = 'binary_target_variable'

# 分析用データセットの準備
analysis_dataset = prepared_dataset.dropna().reset_index(drop=True)
X = analysis_dataset.drop(binary_target, axis=1)
y = analysis_dataset[binary_target].astype(int)

# データセットをトレーニングセットとバリデーションセットに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost分類器のトレーニング
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgb_classifier.fit(X_train, y_train)

# 特徴量重要度のプロット
fig, ax = plt.subplots(figsize=(10, 12))
xgb.plot_importance(xgb_classifier, ax=ax, importance_type='gain', show_values=True, title='Feature Importance')
plt.show()


In [ ]:
#このコードは、複数の患者報告結果(PRO)指標に対して、
#特定の生理的特徴量（ここではanbai特徴量として抽象化）を追加していく過程での
#AUC（エリア・アンダー・ザ・カーブ）の平均値と標準誤差（SE）を計算し、
#それらの結果を格納する複数のDataFrameを作成するプロセスを示しています。
#具体的な変数名やデータセット名を一般化し、データ分析の結果を整理し、可視化する方法を示します。
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

# 各指標（PRO）に対する生理的特徴量（anbai特徴量）の影響を評価
pro_indices = ['PRO1', 'PRO2', 'PRO3']  # PRO指標のリスト
pro_indices_renamed = ['PRO1 renamed', 'PRO2 renamed', 'PRO3 renamed']  # 変更後のPRO指標名のリスト
features = ['feature1', 'feature2', 'feature3']  # 生理的特徴量のリスト

# 結果を格納するDataFrameの準備
result_1 = pd.DataFrame(index=pro_indices_renamed, columns=range(1, len(features) + 1)).fillna(0)
result_2 = pd.DataFrame(index=pro_indices_renamed, columns=range(1, len(features) + 1)).fillna(0)
result_3 = pd.DataFrame(index=pro_indices_renamed, columns=['mean', 'SE', 'Feature Count'] + [f'Feature {i+1}' for i in range(len(features))]).fillna('NA')
result_4 = pd.DataFrame(index=pro_indices_renamed, columns=[f'Feature {i+1}' for i in range(len(features))]).fillna('NA')

# 分析処理（例示）
for pro_index in pro_indices:
    # ここで各PRO指標についてのAUC計算や特徴量の重要度評価などの分析を行う
    # 分析結果をresult_1, result_2, result_3, result_4に格納する
    pass

# 分析結果の可視化や保存処理
# 例: 特徴量ごとのAUC平均値のプロット
for pro_index_renamed in pro_indices_renamed:
    plt.plot(result_1.loc[pro_index_renamed], label=pro_index_renamed)

plt.xlabel('Number of Features')
plt.ylabel('AUC Mean')
plt.title('AUC Mean by Number of Features for Each PRO')
plt.legend()
plt.show()

In [ ]:
#(自分のコメント)この後、特徴量を重要度順に並べて(例: A, B, C, D...)
#インプットデータをA→A+B→A+B+C→...と増やしていき、あるラインを超えると不要な特徴量が多すぎて予測精度が下がるので
#そこでデータの追加をストップして最もAUCが高い特徴量の組み合わせを求めていました。
#理論上は特徴量の組み合わせは2^(特徴量の種類)だけあるので全部試せたら良いのですが、
#マシンのスペック的に難しかったので重要度順に並べることで簡易的に最適な特徴量の組み合わせを実装していました。
#ChatGPTに変換をお願いしたところ、コードが長すぎて下のようにかなり省略しすぎた形になってしまったので、何をしたのかだけここまでの説明で掴んでいただけると幸いです。

import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import RepeatedKFold, train_test_split
import numpy as np
from tqdm import tqdm

# PRO指標のリスト
pro_list = ['PRO1', 'PRO2', 'PRO3']

# 分析と結果保存の設定
n_splits = 5
n_repeats = 100
folder_id = 'your_folder_id_here'  # Google Drive内の保存先フォルダID

# 特徴量の重要度とAUC評価の分析
for pro in pro_list:
    # 分析用データセットの準備
    target = pro
    dataset = prepare_dataset(pro)  # データセット準備関数は仮想的なものです
    X, y = dataset.drop(target, axis=1), dataset[target]

    # 特徴量選択とモデル評価を繰り返し実行
    auc_results = evaluate_features(X, y, n_splits, n_repeats)  # 特徴量評価関数は仮想的なものです

    # Google Driveに結果を保存
    save_to_drive(auc_results, pro, folder_id)  # 結果保存関数は仮想的なものです

def prepare_dataset(pro):
    # データセットの準備に関するコード
    return dataset

def evaluate_features(X, y, n_splits, n_repeats):
    # 特徴量の重要度とAUCの評価に関するコード
    return auc_results

def save_to_drive(data, pro, folder_id):
    # Google Driveにデータを保存するコード
    pass
